In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

Загружаем векторы

In [2]:
vectors = {
    "mus": "../data/mus_vectors.csv",
    "auto": "../data/auto_vectors.csv"
}

Отзывы об автомобилях примем за исходный домен, а отзывы о музыкальных инструментах - за целевой 

In [3]:
def load_vectors(path: str, dummy=False) -> pd.DataFrame:
    if not dummy:
        return pd.read_csv(path, sep="\t")
    else:
        first_line_skipped = False
        data = {
            "overall": [],
            "vectors": []
        }
        cntr = 1
        with open(path) as file:
            for line in file:
                clear_output(True)
                display("Reading line %d" % cntr)
                if not first_line_skipped:
                    first_line_skipped = True
                    continue
                score, vectors = line.split("\t")
                vector = eval(vectors)
                if type(vector) != type([]):
                    raise ValueError("{} in line {}".format(vector, cntr))
                data["overall"].append(eval(score))
                data["vectors"].append(vector)
                cntr += 1
        return pd.DataFrame(data)

In [4]:
mus_df = load_vectors(vectors["mus"])

In [5]:
# Кернель умирает при загрузке большого файла,
# Поэтому грузим вручную
auto_df = load_vectors(vectors["auto"], dummy=True)

'Reading line 20473'

Преобразуем векторы из строк в списки. По какой-то причине Series.apply наглухо стопорит кернель

In [6]:
for i in range(mus_df.shape[0]):
    clear_output(True)
    display("{} / {}".format(i+1, mus_df.shape[0]))
    mus_df.at[i, "vectors"] = eval(mus_df.loc[i, "vectors"])

'10261 / 10261'

In [7]:
# Кодируем классы. Если оценка больше 3, то считаем ее хорошей
# Иначе отзыв плохой
auto_df["overall"] = (auto_df["overall"] > 3).astype(int)
mus_df["overall"] = (mus_df["overall"] > 3).astype(int)

Обучаем LSTM-автоэнкодер на данных из двух доменов

In [8]:
X_train = pd.concat([auto_df["vectors"], mus_df["vectors"]]).values

In [9]:
X_train.shape

(30734,)

In [10]:
# Converting to ndarray
for i in range(X_train.shape[0]):
    clear_output(True)
    display("{} / {}".format(i+1, X_train.shape[0]))
    X_train[i] = np.array([np.array(vec) for vec in X_train[i]])

'30734 / 30734'

In [11]:
np.random.shuffle(X_train)

In [12]:
def data_generator():
    while True:
        x = X_train[np.random.choice(np.arange(X_train.shape[0]))]
        x = np.array([x])
        # Попытка поймать баг
        if len(x.shape) != 3:
            print(x)
        yield x, x 

In [13]:
train_percent = 0.7

In [14]:
def labeled_data_generator():
    while True:
        index = np.random.choice(np.arange(auto_df.shape[0] * train_percent))
        x = auto_df.loc[index, "vectors"]
        x = np.array([x])
        y = auto_df.loc[index, "overall"]
        y = np.array([y]).reshape([1, 1, 1])
        yield x, y

In [15]:
def auto_val_data_generator():
    while True:
        index = np.random.choice(np.arange(auto_df.shape[0])[int(auto_df.shape[0] * train_percent): ])
        x = auto_df.loc[index, "vectors"]
        x = np.array([x])
        y = auto_df.loc[index, "overall"]
        y = np.array([y]).reshape([1, 1, 1])
        yield x, y

Используем pretraining.
Из-за того, что размер последовательностей не фиксирован, а батч для обучения должен быть тензором, приходится обучать модель по одному сэмплу за раз.

In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 128
num_classes = 2
latent_space_dim = 32

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(latent_space_dim, return_sequences=True,
               input_shape=(None, data_dim)))
model.add(LSTM(128, return_sequences=True))
# model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='mean_squared_error',
              optimizer='adagrad')
model.fit_generator(data_generator(), steps_per_epoch=5000, epochs=10, verbose=1)

Using TensorFlow backend.


Epoch 1/10
5000/5000 [==============================] - 207s 41ms/step - loss: 1.5066
Epoch 2/10
5000/5000 [==============================] - 199s 40ms/step - loss: 1.3660
Epoch 3/10
5000/5000 [==============================] - 198s 40ms/step - loss: 1.3310
Epoch 4/10
5000/5000 [==============================] - 201s 40ms/step - loss: 1.3113
Epoch 5/10
5000/5000 [==============================] - 208s 42ms/step - loss: 1.2957
Epoch 6/10
5000/5000 [==============================] - 201s 40ms/step - loss: 1.2847
Epoch 7/10
5000/5000 [==============================] - 203s 41ms/step - loss: 1.2805
Epoch 8/10
5000/5000 [==============================] - 196s 39ms/step - loss: 1.2706
Epoch 9/10
5000/5000 [==============================] - 191s 38ms/step - loss: 1.2647
Epoch 10/10
5000/5000 [==============================] - 206s 41ms/step - loss: 1.2524


На всякий случай сохраняем модель

In [17]:
model.save("./lstm_v1.hdf5")

Удаляем последний слой

In [18]:
model.layers.pop()
model.layers

Добавляем слой для классификации

In [19]:
model.add(Dense(1, activation="sigmoid"))

In [20]:
model.layers

In [21]:
model.compile(loss='binary_crossentropy',
              optimizer='adagrad', metrics=["accuracy"])

In [22]:
model.fit_generator(labeled_data_generator(), steps_per_epoch=5000, validation_steps=1000,
                    epochs=10, verbose=1, validation_data=auto_val_data_generator())

Epoch 1/10
5000/5000 [==============================] - 219s 44ms/step - loss: 0.3274 - acc: 0.8877 - val_loss: 0.3825 - val_acc: 0.8456
Epoch 2/10
5000/5000 [==============================] - 203s 41ms/step - loss: 0.3196 - acc: 0.8800 - val_loss: 0.4083 - val_acc: 0.8354
Epoch 3/10
5000/5000 [==============================] - 187s 37ms/step - loss: 0.3177 - acc: 0.8779 - val_loss: 0.3313 - val_acc: 0.8754
Epoch 4/10
5000/5000 [==============================] - 191s 38ms/step - loss: 0.3130 - acc: 0.8797 - val_loss: 0.3553 - val_acc: 0.8607
Epoch 5/10
5000/5000 [==============================] - 186s 37ms/step - loss: 0.3118 - acc: 0.8780 - val_loss: 0.3868 - val_acc: 0.8393
Epoch 6/10
5000/5000 [==============================] - 187s 37ms/step - loss: 0.2973 - acc: 0.8848 - val_loss: 0.3420 - val_acc: 0.8672
Epoch 7/10
5000/5000 [==============================] - 188s 38ms/step - loss: 0.2951 - acc: 0.8847 - val_loss: 0.3908 - val_acc: 0.8343
Epoch 8/10
5000/5000 [===================

Таким образом, точность предобученной модели на исходном домене составила 0.88
Теперь проведем валидацию модели на объектах из целевого домена

In [23]:
def mus_test_set():
    while True:
        ind = np.random.choice(np.arange(mus_df.shape[0]))
        x = mus_df.loc[ind, "vectors"]
        x = np.array([x])
        y = mus_df.loc[ind, "overall"]
        y = np.array([y]).reshape([1, 1, 1])
        yield x, y

def test_target_domain(size: int, model):
    acc = model.evaluate_generator(mus_test_set(), steps=size)
    print("Target domain test accuracy: {}".format(acc[1]))
    
def test_source_domain(size: int, model):
    acc = model.evaluate_generator(auto_val_data_generator(), steps=size)
    print("Source domain test accuracy: {}".format(acc[1]))

In [92]:
samples = 5000
test_source_domain(samples, model)
test_target_domain(samples, model)

Source domain test accuracy: 0.8306105344431475
Target domain test accuracy: 0.8749965798421298


Сохраняем модель

In [25]:
model.save("./lstm_dense_v1.hdf5")

Точность достигла хороших значений для обоих доменов.
> Source domain test accuracy: 0.8351154837766662<br>
> Target domain test accuracy: 0.8674490487938746 

Возможно, это вызвано дисбалансом классов и высокой полнотой. Нужно проверить сбалансированную метрику accuracy

In [95]:
from sklearn.metrics import accuracy_score

def check_balanced_accuracy(n_samples: int) -> None:
    # Checking source domain 
    classes = auto_df.groupby("overall").count()
    zero_weight = classes.loc[classes.index==0].values[0][0] / auto_df.shape[0]
    one_weight = classes.loc[classes.index==1].values[0][0] / auto_df.shape[0]
    index = np.random.choice(np.arange(auto_df.shape[0])[int(auto_df.shape[0] * train_percent): ], 
                             size=n_samples)
    x = auto_df.loc[index, "vectors"].values
    x = [np.array([i]) for i in x]
    y_true = auto_df.loc[index, "overall"].values
    y_pred = []
    sample_weights = y_true.astype("float32")
    # для каждой единицы для баланса присваиваем вес нуля
    sample_weights *= zero_weight
    # а для каждого нуля - больший вес, вес единицы
    sample_weights[sample_weights == 0] += one_weight
    for sample in x:
        y_pred.append(np.round(np.mean(model.predict_classes(sample))))
    y_pred = np.array(y_pred)
    score = accuracy_score(y_true, y_pred, sample_weight=sample_weights)
    print("Source domain test balanced accuracy: {}".format(score))
    
    # Checking target domain
    classes = mus_df.groupby("overall").count()
    zero_weight = classes.loc[classes.index==0].values[0][0] / auto_df.shape[0]
    one_weight = classes.loc[classes.index==1].values[0][0] / auto_df.shape[0]
    index = np.random.choice(np.random.choice(np.arange(mus_df.shape[0])), 
                             size=n_samples)
    x = mus_df.loc[index, "vectors"].values
    x = [np.array([i]) for i in x]
    y_true = mus_df.loc[index, "overall"].values
    y_pred = []
    sample_weights = y_true.astype("float32")
    # для каждой единицы для баланса присваиваем вес нуля
    sample_weights *= zero_weight
    # а для каждого нуля - больший вес, вес единицы
    sample_weights[sample_weights == 0] += one_weight
    for sample in x:
        y_pred.append(np.round(np.mean(model.predict_classes(sample))))
    y_pred = np.array(y_pred)
    score = accuracy_score(y_true, y_pred, sample_weight=sample_weights)
    print("Target domain test balanced accuracy: {}".format(score))
    
check_balanced_accuracy(5000)

Source domain test balanced accuracy: 0.5861532156295663
Target domain test balanced accuracy: 0.5921612097759897


Теперь мы получили более правдоподобные значения:

>Source domain test balanced accuracy: 0.5861532156295663<br>
Target domain test balanced accuracy: 0.5921612097759897

Тем не менее, они довольно близки. Можно сказать, что baseline-модель успешно решает задачу domain adaptation

In [89]:
mus_df.groupby("overall").count()

,vectors
overall,
0,1239
1,9022


In [70]:
auto_df.groupby("overall").count()

,vectors
overall,
0,2578
1,17895
